In [ ]:
# 2. Modelo híbrido con PyTorch
class HybridModel(nn.Module):
    def __init__(self, metadata_input_size):
        super(HybridModel, self).__init__()
        self.image_model = models.efficientnet_b0(weights=EfficientNet_B0_Weights.DEFAULT)
        self.image_model.classifier = nn.Sequential(
            nn.Linear(self.image_model.classifier[1].in_features, 64), #128
            nn.ReLU(),
            #nn.Dropout(0.3) #Sin Drop out funciona mejor
        )

        self.metadata_fc = nn.Sequential(
            nn.Linear(metadata_input_size, 128),
            nn.ReLU(),
            nn.Dropout(0.2) #0.3, 0.2,0.1 0,2 es el mejor valor
        )

        self.combined_fc = nn.Sequential(
            nn.Linear(192, 64), #256
            nn.ReLU(),
            nn.Dropout(0.2),#0.3, 0.2, 0.1. 0,2 es el mejor valor
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, image, metadata):
        image_features = self.image_model(image)
        metadata_features = self.metadata_fc(metadata)
        combined = torch.cat((image_features, metadata_features), dim=1)
        output = self.combined_fc(combined)
        return output

In [ ]:
#Punto 2 version 2.1
class HybridModel(nn.Module):
    def __init__(self, metadata_input_size):
        super(HybridModel, self).__init__()
        self.image_model = models.efficientnet_b0(weights=EfficientNet_B0_Weights.DEFAULT)

        # Modificar la capa de entrada para aceptar imágenes de 128x128
        self.image_model.features[0][0] = nn.Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

        self.image_model.classifier = nn.Sequential(
            nn.Linear(self.image_model.classifier[1].in_features, 64),
            nn.ReLU(),
        )

        self.metadata_fc = nn.Sequential(
            nn.Linear(metadata_input_size, 128),
            nn.ReLU(),
            nn.Dropout(0.2)
        )

        self.combined_fc = nn.Sequential(
            nn.Linear(192, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, image, metadata):
        image_features = self.image_model(image)
        metadata_features = self.metadata_fc(metadata)
        combined = torch.cat((image_features, metadata_features), dim=1)
        output = self.combined_fc(combined)
        return output

In [ ]:
#Aplicando cambios anteriores más finetuning

class HybridModel(nn.Module):
    def __init__(self, metadata_input_size):
        super(HybridModel, self).__init__()
        # Cargar EfficientNet-B0 con pesos preentrenados
        self.image_model = models.efficientnet_b0(weights=EfficientNet_B0_Weights.DEFAULT)

        # Modificar la capa inicial para aceptar imágenes de tamaño 128x128
        self.image_model.features[0][0] = nn.Conv2d(
            in_channels=3, out_channels=32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False
        )

        # Ajustar el clasificador para reducir la dimensionalidad
        self.image_model.classifier = nn.Sequential(
            nn.Linear(self.image_model.classifier[1].in_features, 64),
            nn.ReLU(),
        )

        # Bloque para procesar los metadatos
        self.metadata_fc = nn.Sequential(
            nn.Linear(metadata_input_size, 128),
            nn.ReLU(),
            nn.Dropout(0.2)
        )

        # Capa combinada para salida final
        self.combined_fc = nn.Sequential(
            nn.Linear(192, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

        # Fine-tuning: Congelar las capas iniciales de EfficientNet
        for param in self.image_model.features.parameters():
            param.requires_grad = False

        # Descongelar el clasificador para permitir entrenamiento
        for param in self.image_model.classifier.parameters():
            param.requires_grad = True

    def forward(self, image, metadata):
        # Extracción de características de la imagen
        image_features = self.image_model(image)
        # Procesamiento de los metadatos
        metadata_features = self.metadata_fc(metadata)
        # Combinación de ambas características
        combined = torch.cat((image_features, metadata_features), dim=1)
        # Predicción final
        output = self.combined_fc(combined)
        return output